# 01. Google BigQuery

<br>

### 관계형 데이터베이스 관리 시스템

- OLTP DB의 주요 장점중 하나는 SQL을 지원
- SQL은 DB 컬럼에서 로우 데이터를 가져오는 것뿐만 아니라, 그 이상의 작업이 가능
- 이런 쿼리를 OLTP DB에서 수행한은 것은 매우 비효율적
    - OLTP DB는 데이터 기록과 동시에 읽을 수 있도록, 데이터 일관성을 확보하는 데 최적화 되어있음 $\rightarrow$ 데이터 무결성을 유지하기 위해 데이터를 매우 조심스럽게 잠그기 때문
    
        - `A`컬럼을 효율적으로 필터링하려면, `A`컬럼에 인덱스를 생성해야 하면, `A`컬럼에 인덱스를 생성해야만 DB가 스토리지에서 검색을 최적화하는 작업을 실행
    
    - 읽기속도가 높아지면 쓰기 속도가 느려지는 트레이드오프가 존재
    
        - `A`컬럼에 인덱스를 생성했더라도, 특정 쿼리는 집계, 그룹화 및 정렬 때문에 상당히 느려짐
        
    - OLTP DB는 전체 데이터 셋을 홅어야 하는 애드훅 쿼리 실행에는 적합하지 않음
        - 애드훅 쿼리 : 인덱스 같은 기능을 사용하여 최상의 성능을 발휘하도록 미리 만들어 둔 DB에 쿼리를 하는 것이 아니라 필요에 따라 (성능이 조금 떨어져도) 즉석에서 작성해 실행하는 쿼리
        
<br>

### 맵리듀스 프레임워크
- OLTP DB는 전체 데이터셋의 수회가 필요한 애드훅 쿼리에 적합하지 않아서, 순회가 필요한 특수 목적의 분석은 파이썬 같은 언어로 수행
- 2003년 제프 딘과 산자이 게마와트는 구글 동료들이 대용량 로우 데이터를 처리하기 위해 수백 개의 특수 목적의 연산을 구현에 대해서,
    
    특수 목적의 연산을 두 단계로 추상화하여 복잡도를 낮춤
    
    1. 키/값 쌍을 처리해 주는 중간 키/값 쌍을 생성하는 `map`함수
    2. 동일한 중간 키와 연관된 모든 중간 값을 병합하는 `reduce` 함수
    
    $\rightarrow$  MapReduce는 하둡 (Apache Hadoop)으로 연결
    
<br>

- 하둡은 주보 자바로 빌드된 라이브러리로 시작했지만, 현재 일반적으로 아파치 스파크 (Apache Spark)로 하둡 클러스터에 대한 맞춤 분석을 수행
    - 스파크는 분산된 데이터셋에서 애드훅 SQL쿼리를 실행하는 기능도 존재
    
- HDFS에서의 데이터 파이프라인
    1. HDFS에서 CSV 텍스트 파일로 데이터를 정기적으로 내보냄
    2. 애드훅 분석을 위해서 다음을 수행하는 스파크 프로그램을 작성
        - 텍스트 파일의 데이터를 데이어프레임으로 로드
        - 테이블 이름이 데이터프레임 이름으로 바뀌는 것을 제외하고, SQL쿼리를 실행
        - 결과를 다시 텍스트 파일로 내보냄
    3. 하둡 클러스터에서 스파크 프로그램을 실행

<br>

- HDFS에 데이터를 저장하려면 클러스터가 충분히 커야 함
- 맵리듀스 아키텍처에서는 일반적으로 컴퓨팅 노드가 로컬에 있는 데이터에 접근해야 하며, HDFS는 클러스터의 컴퓨팅 노드에 샤딩(Sharding)되어야 함
- 데이터의 크기와 분석의 필요성은 서로와는 무관하지만, 양쪽 모두 크게 증가 $|rightarrow$ 클러스터가 부족한 경우나 과도하게 프로비저닝 되는 경우가 존재


<br>

### BigQuery : 서버리스, 분산 SQL 엔진
- 빅쿼리는 인프라스트럭처를 운영하지 않아도 RDBMS처럼 SQL쿼리를 실행할 수 있고, 맵리듀스처럼 전체 데이터셋 탐색을 효율적으로 분산할 수 있음
- 빅쿼리는 서버리스 서비스이므로, 인프라를 관리할 필요 없이 쿼리를 실행할 수 있음

<br>

#### [서비스 키 생성](https://support.oqupie.com/portal/9/article/44821)

In [10]:
import glob
from google.cloud import bigquery
from google.oauth2 import service_account
import warnings
warnings.filterwarnings('ignore')

In [11]:
key_path = glob.glob("./*.json")[0]

In [13]:
credentials = service_account.Credentials.from_service_account_file(key_path)

In [14]:
client = bigquery.Client(credentials = credentials, project = credentials.project_id)

In [17]:
sql = f"""
    SELECT 
      EXTRACT(YEAR FROM starttime) AS year,
      EXTRACT(MONTH FROM starttime) AS month, 
      COUNT(starttime) AS number_one_way
    FROM
      `bigquery-public-data.new_york_citibike.citibike_trips`
    WHERE
      start_station_name != end_station_name
    GROUP BY year, month
    ORDER BY year ASC, month ASC
"""

In [18]:
query_job = client.query(sql)
df = query_job.to_dataframe()

In [20]:
df.head()

,year,month,number_one_way
0,2013,7,815324
1,2013,8,970474
2,2013,9,1007799
3,2013,10,1015801
4,2013,11,663544


<br>

### 빅쿼리 작업
- 빅쿼리는 어느정도 중앙 집중화가 되어있고, 보편성을 가지는 DW
- 빅쿼리의장점은 다른 소스의 데이터셋을 조인하거나, 빅쿼리 외부에 저장된 데이터에 쿼리할 때 유용

<br>

#### 여러 데이터셋에 대한 접근

In [23]:
sql = f"""
WITH 
    bicycle_rentals AS (
        SELECT
            COUNT(starttime) as num_trips,
            EXTRACT(DATE from starttime) as trip_date
        FROM `bigquery-public-data.new_york_citibike.citibike_trips`
        GROUP BY trip_date
),

    rainy_days AS (
        SELECT
            date,
            (MAX(prcp) > 5) AS rainy
        FROM (
            SELECT
                wx.date AS date,
                IF (wx.element = 'PRCP', wx.value/10, NULL) AS prcp
            FROM
               `bigquery-public-data.ghcn_d.ghcnd_2016` AS wx
          WHERE
            wx.id = 'USW00094728'
        )
        GROUP BY date
    )

    SELECT
      ROUND(AVG(bk.num_trips)) AS num_trips,
      wx.rainy
    FROM bicycle_rentals AS bk
    JOIN rainy_days AS wx
    ON wx.date = bk.trip_date
    GROUP BY wx.rainy
"""

In [24]:
query_job = client.query(sql)
df = query_job.to_dataframe()

In [25]:
df

,num_trips,rainy
0,39107.0,False
1,32052.0,True


<br>

### ETL, EL, ELT
- DW를 사용하는 전통적인 방법은 ETL (Extract ,Transform, Load) 프로세스
    - 소스에서 로우 데이터를 추출하고, 변환한 후, DW로 로드


- 워크플로우


| 워크플로우 | 구조 | 사용해야 하는 경우 |
| :---- | :- | :- |
| EL | - 구글 클라우드 스토리지의 파일에서 데이를 추출 <br> - 빅쿼리의 기존 스토리지에 불러옴 <br> - 클라우드 컴포져, 클라우드 펑션 또는 예약 쿼리에서 실행 | - 히스토리 데이터를 배치 단위로 불러오는 경우, <br> - 정기적으로 로그 파일을 불러오는 경우 (예: 하루에 한번) |
| ETL | - 펍/섭(Pub/Sup), 구글 클라우드 스토리지, 클라우드 스패너, 클라우드 SQL 등에서 데이터를 추출 <br> - 클라우드 데이터플로우로 데이터를 변환 <br> - 데이터플로우를 파이프라인을 통해 빅쿼리 테이블에 저장 | - 로우 데이터를 빅쿼리에 불러오기 전에 품질 관리, 변환 또는 보강 해야 하는 경우 <br> - 데이터 불러오기가 지속적으로 필요한 경우, 즉 스트리밍이 필요한 경우 <br> - CI/CD(Continouous Integration/Continuous Delivery) 시스템과 통합하고 모든 구성 요소에서 장치 테스트를 수행하려는 경우 |
| ELT | -구글 클라우드 스토리지 파일에서 데이터를 추출 <br> - 빅쿼리에서 원시 형식으로 데이터를 저장 <br>  - 빅쿼리 뷰로 데이터를 즉시 변환 | - 데이터를 사용하려면 어떤 종류의 변환이 필요한지 아직 모르는 실험 데이터셋 <br> - 변환이 QL로 표현될 수 있는 모든 프로덕션 데이터 셋 |

<br>

### 분석
- 빅쿼리는 BI툴과의 연계가 가능
- 빅쿼리는 배열 및 복잡한 조인을 포함한 SQL:2011의 모든 기능을 지원
    - 배열 지원으로 중첩되거나 반복되는 필드를 평평하게 할 필요 없이, 계층 데이터 (JSON 레코드 등)를 빅쿼리에 저장할 수 있음
    
- 두 테이블을 거리나 겹침 조건 (overlap criterio)를 기준으로 조인하는 기능을 포함하여,
    
    위치 인식(location-aware)쿼리를 지원하는 수많은 공간 함수를 제공하는 호가장 기능도 존재
    
- 빅쿼리의 확장 기능을 통하여, 빅쿼리에서 데이터를 내보내지 않고도 모델을 학습하고 예측할 수 있음

- 빅쿼리는 배체 데이터와 스트리밍 데이터 수집 모두 지원
    - REST API를 통해 빅쿼리로 직접 데이터를 스트리밍 할 수 있음

<br>

### bq 명령줄 도구(command-line) 설치
- Google Cloud SDK 설치시 사용 가능
- [Installing Google Cloud SDK](https://cloud.google.com/sdk/docs/install?hl=ko)로 이동
- 윈도우 : Cloud SDK 설치 프로그램 다운로드 또는 파워셀에서 아래 명령어 실행

```Windows
 (New-Object Net.WebClient).DownloadFile("https://dl.google.com/dl/cloudsdk/channels/rapid/GoogleCloudSDKInstaller.exe", "$env:Temp\GoogleCloudSDKInstaller.exe")
 & $env:Temp\GoogleCloudSDKInstaller.exe
```


- gcloud 초기화
```Windows
gcloud init
```